## Etapa 0 — Setup de Dependências e Bibliotecas

Antes de executar o pipeline do *modelo campeão*, garantimos que todas as bibliotecas e recursos estejam disponíveis.  
Este bloco prepara o ambiente, faz downloads necessários (`nltk` e `SpaCy`), carrega o modelo de linguagem em português e combina as listas de stopwords `nltk` + `SpaCy`.



In [1]:
# 🔧 ETAPA: SETUP DE DEPENDÊNCIAS E BIBLIOTECAS

# Dependências padrão
import pandas as pd
import numpy as np
import string
import re
import os

# Instalar unidecode se necessário
try:
    import unidecode
except ImportError:
    !pip install unidecode
    import unidecode

# NLTK
import nltk
from nltk.corpus import stopwords

# SpaCy
import spacy

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, confusion_matrix

# Baixar stopwords PT
nltk.download('stopwords')

# SpaCy pt_core_news_sm
try:
    nlp = spacy.load('pt_core_news_sm')
except:
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "pt_core_news_sm"])
    nlp = spacy.load('pt_core_news_sm')

# Combinar stopwords NLTK + SpaCy
stopwords_pt = set(stopwords.words('portuguese'))
stopwords_spacy = nlp.Defaults.stop_words
combined_stopwords = stopwords_pt.union(stopwords_spacy)

print(f"Stopwords combinadas: {len(combined_stopwords)} termos")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 17.2 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Stopwords combinadas: 500 termos


## Etapa 1 — Pipeline do Modelo Campeão com Lematização, Placeholders e Vetorização BoW

Este bloco executa o pipeline do *modelo campeão* de forma **portável** e **rastreável**, incluindo:
- **Carregamento do dataset** pela URL oficial,
- **Pré-processamento** com substituição inteligente de placeholders (`<DATE>`, `<NUMBER>`, `<PII>`),
- **Tokenização e lematização** com SpaCy,
- **Remoção de stopwords combinadas** (`nltk` + `SpaCy`),
- **Barra de progresso `tqdm`** para monitorar o avanço da lematização,
- **Montagem do texto final** para vetorização,
- Vetorização com **CountVectorizer (unigrama)**,
- **Divisão treino/teste** estratificada com `random_state=42`,
- Treinamento do **LogisticRegression**,
- Avaliação com **classification_report**, **F1 Score weighted** e matriz de confusão.



In [3]:
# 🔧 ETAPA: PIPELINE COMPLETO DO MODELO **CAMPEÃO**
# 1️⃣ Carregar dataset
url = "https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv"
df = pd.read_csv(url, sep=';')
print(df.head(5))

# 2️⃣ Limpeza básica
df.dropna(subset=['descricao_reclamacao'], inplace=True)

# 3️⃣ Substituição inteligente de placeholders
def replace_placeholders(text):
    text = re.sub(r'\b\d{2}/\d{2}/\d{4}\b', '<DATE>', text)
    text = re.sub(r'\b\d{2}-\d{2}-\d{4}\b', '<DATE>', text)
    text = re.sub(r'\b\d{4}\b', '<YEAR>', text)
    text = re.sub(r'\b\d+\b', '<NUMBER>', text)
    text = re.sub(r'X{2,}', '<PII>', text, flags=re.IGNORECASE)
    return text

# 4️⃣ Função de pré-processamento + lematização
def preprocess_and_lemmatize(text):
    text = replace_placeholders(text)
    text = text.lower()
    text = unidecode.unidecode(text)
    text = re.sub(r'\s+', ' ', text).strip()
    doc = nlp(text)
    tokens = [
        token.lemma_ for token in doc
        if token.is_alpha and token.lemma_ not in combined_stopwords
    ]
    return tokens

# 5️⃣ Usar tqdm para progresso
from tqdm.notebook import tqdm
tqdm.pandas()

df['tokens_lematizados'] = df['descricao_reclamacao'].progress_apply(preprocess_and_lemmatize)

print(df[['descricao_reclamacao', 'tokens_lematizados']].head(5))

# 6️⃣ Texto final para vetorização
df['texto_final'] = df['tokens_lematizados'].apply(lambda tokens: " ".join(tokens))

# 7️⃣ Vetorização BoW
vectorizer = CountVectorizer(ngram_range=(1,1))
X = vectorizer.fit_transform(df['texto_final'])
y = df['categoria']

print(f"Shape matriz vetorial: {X.shape}")

# 8️⃣ Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")

# 9️⃣ Treinar modelo
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# 🔟 Avaliar com apresentação formatada
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import pandas as pd
from IPython.display import display

y_pred = clf.predict(X_test)

# Report
report = classification_report(y_test, y_pred, target_names=clf.classes_, digits=2)
print("\n🔎 **Classification Report**:\n")
print(report)

# F1 Score weighted
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"\n✅ **F1 Score (weighted): {f1:.2%}**")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
labels = clf.classes_
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

print("\n🔍 **Matriz de Confusão:**")
display(cm_df)


   id_reclamacao              data_abertura  \
0        3229299  2019-05-01T12:00:00-05:00   
1        3199379  2019-04-02T12:00:00-05:00   
2        3233499  2019-05-06T12:00:00-05:00   
3        3180294  2019-03-14T12:00:00-05:00   
4        3224980  2019-04-27T12:00:00-05:00   

                             categoria  \
0              Hipotecas / Empréstimos   
1  Cartão de crédito / Cartão pré-pago   
2  Cartão de crédito / Cartão pré-pago   
3  Cartão de crédito / Cartão pré-pago   
4           Serviços de conta bancária   

                                descricao_reclamacao  
0  Bom dia, meu nome é xxxx xxxx e agradeço se vo...  
1  Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...  
2  O cartão Chase foi relatado em xx/xx/2019. No ...  
3  Em xx/xx/2018, enquanto tentava reservar um ti...  
4  Meu neto me dê cheque por {$ 1600,00} Eu depos...  


  0%|          | 0/21072 [00:00<?, ?it/s]

                                descricao_reclamacao  \
0  Bom dia, meu nome é xxxx xxxx e agradeço se vo...   
1  Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...   
2  O cartão Chase foi relatado em xx/xx/2019. No ...   
3  Em xx/xx/2018, enquanto tentava reservar um ti...   
4  Meu neto me dê cheque por {$ 1600,00} Eu depos...   

                                  tokens_lematizados  
0  [dia, nome, pii, pii, agradeco, voce, puder, a...  
1  [atualizei, cartao, pii, pii, informar, por o,...  
2  [cartao, chase, relatar, em o, entanto, pedido...  
3  [reservar, ticket, pii, pii, deparar, oferta, ...  
4  [neto, cheque, depositei, em o, conta, chase, ...  
Shape matriz vetorial: (21072, 29483)
X_train: (15804, 29483), X_test: (5268, 29483)

🔎 **Classification Report**:

                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.91      0.92      0.91      1252
            Hipotecas / Empréstimos       0.91      0.91      

,Cartão de crédito / Cartão pré-pago,Hipotecas / Empréstimos,Outros,Roubo / Relatório de disputa,Serviços de conta bancária
Cartão de crédito / Cartão pré-pago,1149,20,21,48,14
Hipotecas / Empréstimos,16,880,14,29,23
Outros,22,23,469,17,27
Roubo / Relatório de disputa,46,30,19,1058,53
Serviços de conta bancária,28,15,15,51,1181
